# Домашнее задание 2

В этом домашнем задании мы поупражняемся с ETS-моделью и STL-разложением. Сначала коротко про данные.

Представьте себе стандартный банкомат, который работает только на выдачу. Его нужно периодически пополнять наличностью, чтобы он функционировал непрерывно. Однако выезд инкассаторов является делом хлопотным и недешёвым, так что хотелось бы не гонять их лишний раз заполнять наполовину полный банкомат. Хотелось бы научиться предсказывать спрос людей на наличность в банкомате, чтобы можно было предсказывать время, когда количество денег в банкомате снизится до минимально допустимой величины. Этим мы и займёмся. Для простоты возьмём один конкретный банкомат. Нам доступны объёмы выдачи в банкомате по дням.

##### 1) Загрузка данных и визуальный анализ (1 балл)

Загрузите данные из файла atm.csv. Рекомендуется для нормальной работы некоторых фукнций перевести индекс в тип PeriodIndex. Например, можно создать date_range с нужной частотой. Изобразите временной ряд, его автокорреляции и частные автокорреляции. Что вы можете сказать о структуре ряда? Присутствует ли в нём тренд? Сезонность? Цикличность? Стационарность?

In [102]:
# ༼ つ ◕_◕ ༽つ

##### 2) Предобработка данных (0.5 балла)

Разбейте данные на трейн и тест. Размер теста: последние 30 дней.

Как легко заметить, данные неотрицательны. Но давайте на случай использования мультипликативных ETS-моделей их децентрируем. К тренировочной части прибавьте произвольную небольшую константу (например 100 рублей) и нормируйте её. Обратите внимание, что обратную нормировку и центрирование придётся производить над прогнозами итоговой модели при подсчёте метрик относительно теста.

Дополнительно можете произвести обратимые преобразования, которые сочтёте необходимыми.

In [105]:
# ༼ つ ◕_◕ ༽つ

## Покомпонентное прогнозирование

В данном разделе мы попробуем пофильтровать ряд с помощью двух моделей: STL-разложения и ETS-модели.

### Блок 1. STL

##### Разложение (0.3 балла)

Начнём с STL. Примените на трейне STL-разложение. В качестве параметра period возьмите самый сильный лаг автокорреляций. Визуализируйте отдельные компоненты.

In [2]:
# ༼ つ ◕_◕ ༽つ

##### Прогнозирование отдельных компонент (0.6 балла)

Возьмём выделенные из STL компоненты. Постройте какие-нибудь простые модели (без фичей) для тренда и сезонности.

Например, для тренда можно взять модель полиномального тренда из sktime. Для сезонности можно использовать NaiveForecaster с указанием периодичности. Спрогнозируйте каждую компоненту 30 шагов (по 0.3 за каждую компоненту).

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### Прогнозирование остатков. 

##### Подготовка данных (0.9 балла)

Достанем временной ряд остатков из STL-разложения. Построим отдельную модель с остатками STL в качестве таргета и потом сложим с прогнозами отдельных компонент из предыдущего шага. Получим этакий бутерброд. Прогнозируем отдельно на 30 шагов каждую компоненту: тренд, сезонность и ошибку, а потом суммируем прогнозы.

Многошаговый прогноз остатков потребует уже модели с фичами. По сути эта задача эквивалентна многошаговому прогнозированию из ДЗ-1. В этот раз вам не будет дано конкретных указаний по генерации переменных. Но будет выдана дополнительная информация о банкомате. Можете её использовать, можете нет, на ваше усмотрение.

Расположение: ул Крымский Вал, 9стр45

Дополнительная информация: ГАУК культуры города Москвы ЦПКиО им. М.Горького

Широта: 55.73259

Долгота: 37.600891

Также в помощь прилагается файл с различными календарными фичами (events.csv), чтобы не повторять в этой домашке то же самое, что в прошлой. Можете использовать любые фичи из этого файла, но для засчитывания пункта необходимо придумать минимум три фичи (по 0.3 за каждую), не входящие в приложенный файл. Каждую используемую фичу (не важно, из файла или придуманную вами) необходимо кратко обосновать. Не забывайте учитывать потенциальные утечки. За их наличие балл также может быть снижен.

Результатом пункта должны быть вектор y_train и матрица X_train для модели остатков.

In [ ]:
# ༼ つ ◕_◕ ༽つ

#### Кросс-валидация

Теперь займёмся подбором оптимальной модели остатков STL на трейне. Для этого используем кросс-валидацию.

##### Реализация метода (0.6 балла)

Так как нас интересуют именно многошаговые прогнозы, стандартная кросс-валидация нам не подойдёт. Выберите любой из двух методов кросс-валидации временных рядов (скользящее или расширяющееся окно) на ваше усмотрение.

 Реализуйте класс, который будет по входным параметрам окна выдавать индексы всех фолдов. По поведению можно ориентироваться на знакомый вам k-fold. Хорошей практикой будет возвращать в методе split генератор, хотя это и не обязательно. На каждом фолде модель будет обучаться на трейне и делать многошаговый прогноз на тест. Позаботьтесь о том, чтобы ширина окна была соотнесена в логичной пропорции с горизонтом прогнозирования. Ширина окна 35 при горизонте 30 -- сомнительная практика.


In [ ]:
class CustomCrossVal:
    def __init__(self, window_size = 100, step=30):

        '''Args:
            window_size (int): The size of window. For expanding window -- initial size of window
            step (int): Window step. For expanding window is a growth of window. For sliding window is number of obs to slide.
        '''
        pass

    def split(self, X):
        '''Args:
            X (pandas DataFrame): Dataframe to split.
           '''
        pass

##### Отбор моделей (0.5 балла)

Выберите 3-4 модели-кандидата и отберите с помощью реализованной вами кросс-валидации наилучшую. Можно использовать разные модели либо разные наборы фичей. Метрика: MAE.

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Прогноз моделей (0.5 балла)

Обучите наилучшую модель на всём трейне и постройте прогноз на тестовый период. Используйте любую из многошаговых стратегий, но прямая будет самым простым решением. 

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Бэктестинг (1 балл)

Обучив наилучшую модель на трейне, вычтите из y_train тренд и сезонность (полученные из STL), а также прогноз вашей модели отатков на трейн. Таким образом вы получите остатки вашей совокупной модели на трейне. Нарисуйте графики автокорреляций этих остатков. Есть ли в каких-то интересных датах значимые пики? Как думаете, с чем эти ошибки модели связаны? (0.5 балла)

Реализуйте тест Ljung-Box. Протестируйте остатки на наличие серийных автокорреляций при нескольких m. $\alpha = 0.05$. (0.5 балла)

Если гипотеза отвергается, можете попытаться улучшить модель и поискать фичи по графикам автокорреляций, но это опционально. В ряде ситуаций становится сложно построить модель, в которой гипотеза не отвергается, поэтому иногда на наличие автокорреляций закрывают глаза.

In [ ]:
# ༼ つ ◕_◕ ༽つ

Бонус! (0.5 балла)

Реализуйте тест Бройша-Годфри. Описание можно почерпнуть из [Википедии](https://en.wikipedia.org/wiki/Breusch–Godfrey_test). Протестируйте остатки на наличие автокорелляций. Изменился ли результат?

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Результаты и визуализация (0.6 балла)

Соберите в сумму прогнозы всех компонент STL-разложения. Не забудьте на сумму прогнозов компонент применить преобразования, обратные тем, которые были сделаны на трейне.  Визуализируйте прогноз и тестовые данные. Подсчитайте MAE прогноза. Подсчитайте MAE сезонного наивного прогноза (на каждой точке теста -- последнее наблюдение за аналогичный день в предыдущем месяце, 30 дней назад). Справилась ли ваша модель лучше?

In [ ]:
# ༼ つ ◕_◕ ༽つ

### Блок 2. ETS-модель

Теперь повторим то же самое, только моделировать тренд и сезонность будем одновременно одной ETS-моделью, а на остатки ETS-модели навешивать вторую модель.

##### Визуальный анализ (1 балл)

Вернитесь к y_train. Взгляните на ряд и опишите, какие из вариантов ETS-модели подошли бы под наши данные. Сформируйте сетку параметров для моделей-кандидатов. Кратко поясните каждую опцию, которую вы включаете в сетку, основываясь на визуальном анализе.

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Кросс-валидация (0.3 балла)

С помощью кросс-валидации выберите наилучшую по MAE ETS-модель. Обучите её на всём трейне и спрогнозируйте на тест. 

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Подготовка данных (0.1 балла)

Достаньте остатки ETS-модели на трейне. Они будут использованы в качестве таргета для второй модели. Можно использовать те же фичи, что и в разделе про STL, без дополнительных пояснений. Результатом пункта должны быть вектор y_train и матрица X_train для модели остатков.

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Отбор моделей остатков и кросс-валидация (0.5 баллов)

Отберите 3-4 модели-кандидата для прогнозирования остатков и отберите на кросс-валидации наилучшую по MAE модель. Размер валидации на каждом фолде всё также равен 30, как длина теста. Используйте реализованный ранее класс для кросс-валидации. 

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Прогноз остатков ETS (0.5 баллов)
Обучите наилучшую модель на всём трейне. Постройте прогноз на тест с помощью любой многошаговой стратегии. Как и ранее, проще всего использовать прямую.

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Бэктестинг (0.5 балла)

Класс ETS-модели не позволяет напрямую получить прогноз на трейне. Но позволяет сразу получить остатки на трейне. Возьмите эти остатки и вычтите из них прогноз второй модели. Таким образом вы получите финальные остатки вашей двухуровневой модели. Нарисуйте графики автокорреляций этих остатков. Протестируйте их на наличие серийных автокорреляций любым из тестов, реализованных ранее.

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Результаты (0.6 балла)

Постройте суммарный прогноз ETS-модели и модели остатков на тесте. Визуализируйте прогноз и тестовые значения на графике. Подсчитайте MAE прогноза. Добавьте на график прогноз модели из Блока 1. Какая из двух моделей получилась лучше? Как думаете, почему? Обе ли модели смогли пробить порог сезонной наивной модели?

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Скиньте мем (Бесценно)

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Рубрика "как вам домашка?"

Поделитесь, что вам понравилось, а что нет. Это важно